In [27]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import matplotlib.pyplot as plt
from imblearn.under_sampling import NearMiss
from collections import Counter
from sklearn.model_selection import KFold
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "BANKRUPT"]

In [28]:
cleaned_df=pd.read_csv("/Users/monalisa/Downloads/cleaned_data.csv")
cleaned_df.head()

,Unnamed: 0,Tobin's Q,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth,BK
0,1,-0.943577,0.000054,0.368583,0.137898,0.040532,0.009772,0.538394,0.059158,0.003440,-0.022715,0.195206,-0.101516,0.055176,0
1,2,-1.170303,-0.000979,0.232118,0.129773,0.037908,0.007736,0.505855,0.057785,0.003145,-0.022708,0.677631,0.167686,1.208515,0
2,3,-0.675053,-0.000610,0.102753,0.131109,0.037908,0.006040,0.373341,0.058470,0.003178,-0.022577,-0.188691,-0.235070,-0.395276,0
3,4,-0.473397,-0.000573,0.549397,0.129773,0.037908,0.001493,0.297050,0.058470,0.003145,-0.022412,0.426924,0.019069,0.093287,0
4,5,-0.610216,0.000141,0.403827,0.133804,0.039875,0.002573,0.390119,0.059848,0.003276,-0.022454,0.209474,0.090797,0.162478,0


In [29]:
# dropping passed values 
cleaned_df.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [30]:
#Create independent and Dependent Features
columns = cleaned_df.columns.tolist()
# Filter the columns to remove data we do not want 

# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["BK"]]
# Store the variable we are predicting 
target = "BK"
# Define a random state 
state = np.random.RandomState(42)
#X = data[columns]
X = cleaned_df[columns]
print(X.shape)
y=cleaned_df[target]

print(y.shape)

(85972, 13)
(85972,)


In [31]:
#Splitting of dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
from imblearn.combine import SMOTEENN, SMOTETomek
X_train_SMOTEENN, y_train_SMOTEENN = SMOTEENN(random_state=42).fit_sample(X_train,y_train)
#X_val_under, y_val_under = RandomUnderSampler(random_state=42).fit_sample(X_val_std,y_val)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
C = np.logspace(0, 4, 10)
penalty = [ 'l2']
logistic = LogisticRegression(solver='lbfgs', max_iter=10000)
hyperparameters = dict(C=C, penalty=penalty)
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0,scoring='roc_auc')
best_model = clf.fit(X_train_SMOTEENN, y_train_SMOTEENN)
y_pred_Logisticregression=best_model.predict(X_test)





In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_Logisticregression))

              precision    recall  f1-score   support

           0       1.00      0.69      0.81     17084
           1       0.02      0.74      0.03       111

    accuracy                           0.69     17195
   macro avg       0.51      0.71      0.42     17195
weighted avg       0.99      0.69      0.81     17195



In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_Logisticregression)

array([[11726,  5358],
       [   29,    82]])

In [36]:
from sklearn.metrics import accuracy_score,cohen_kappa_score,f1_score,log_loss,roc_auc_score
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred_Logisticregression)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred_Logisticregression)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred_Logisticregression)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred_Logisticregression)))
print("AUC = {:.2f}".format(roc_auc_score(y_test, y_pred_Logisticregression)))

Accuracy = 0.69
Kappa = 0.02
F1 Score = 0.03
Log Loss = 10.82
AUC = 0.71


In [37]:
X_train_SMOTETomek, y_train_SMOTETomek = SMOTETomek(random_state=42).fit_sample(X_train,y_train)

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
C = np.logspace(0, 4, 10)
penalty = ['l2']
logistic = LogisticRegression()
hyperparameters = dict(C=C, penalty=penalty)
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0,scoring='roc_auc')
best_model = clf.fit(X_train_SMOTETomek, y_train_SMOTETomek)
y_pred_Logisticregression=best_model.predict(X_test)

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_Logisticregression))

              precision    recall  f1-score   support

           0       1.00      0.72      0.84     17084
           1       0.02      0.72      0.03       111

    accuracy                           0.72     17195
   macro avg       0.51      0.72      0.44     17195
weighted avg       0.99      0.72      0.83     17195



In [40]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_Logisticregression)

array([[12356,  4728],
       [   31,    80]])

In [41]:
from sklearn.metrics import accuracy_score,cohen_kappa_score,f1_score,log_loss,roc_auc_score
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred_Logisticregression)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred_Logisticregression)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred_Logisticregression)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred_Logisticregression)))
print("AUC = {:.2f}".format(roc_auc_score(y_test, y_pred_Logisticregression)))

Accuracy = 0.72
Kappa = 0.02
F1 Score = 0.03
Log Loss = 9.56
AUC = 0.72


In [42]:
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
X_train_BorderlineSMOTE, y_train_BorderlineSMOTE = BorderlineSMOTE(random_state=42).fit_sample(X_train,y_train)

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
C = np.logspace(0, 4, 10)
penalty = ['l2']
logistic = LogisticRegression()
hyperparameters = dict(C=C, penalty=penalty)
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)
best_model = clf.fit(X_train_BorderlineSMOTE, y_train_BorderlineSMOTE)
y_pred_Logisticregression=best_model.predict(X_test)



In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_Logisticregression))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94     17084
           1       0.03      0.58      0.06       111

    accuracy                           0.89     17195
   macro avg       0.52      0.73      0.50     17195
weighted avg       0.99      0.89      0.94     17195



In [45]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_Logisticregression)

array([[15226,  1858],
       [   47,    64]])

In [46]:
from sklearn.metrics import accuracy_score,cohen_kappa_score,f1_score,log_loss,roc_auc_score
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred_Logisticregression)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred_Logisticregression)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred_Logisticregression)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred_Logisticregression)))
print("AUC = {:.2f}".format(roc_auc_score(y_test, y_pred_Logisticregression)))

Accuracy = 0.89
Kappa = 0.05
F1 Score = 0.06
Log Loss = 3.83
AUC = 0.73


In [22]:
X_train_ADASYN, y_train_ADASYN = ADASYN(random_state=42).fit_sample(X_train,y_train)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
C = np.logspace(0, 4, 10)
penalty = ['l2']
logistic = LogisticRegression()
hyperparameters = dict(C=C, penalty=penalty)
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)
best_model = clf.fit(X_train_ADASYN, y_train_ADASYN)
y_pred_Logisticregression=best_model.predict(X_test)

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_Logisticregression))

              precision    recall  f1-score   support

           0       1.00      0.71      0.83     17084
           1       0.02      0.73      0.03       111

    accuracy                           0.71     17195
   macro avg       0.51      0.72      0.43     17195
weighted avg       0.99      0.71      0.83     17195



In [26]:
from sklearn.metrics import accuracy_score,cohen_kappa_score,f1_score,log_loss,roc_auc_score
print("Accuracy = {:.2f}".format(accuracy_score(y_test, y_pred_Logisticregression)))
print("Kappa = {:.2f}".format(cohen_kappa_score(y_test, y_pred_Logisticregression)))
print("F1 Score = {:.2f}".format(f1_score(y_test, y_pred_Logisticregression)))
print("Log Loss = {:.2f}".format(log_loss(y_test, y_pred_Logisticregression)))
print("AUC = {:.2f}".format(roc_auc_score(y_test, y_pred_Logisticregression)))

Accuracy = 0.71
Kappa = 0.02
F1 Score = 0.03
Log Loss = 9.98
AUC = 0.72


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
C = np.logspace(0, 4, 10)
penalty = ['l2']
logistic = LogisticRegression()
cv = KFold(n_splits=5, shuffle=True, random_state=None)
hyperparameters = dict(C=C, penalty=penalty)
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)
best_model = clf.fit(X_train, y_train)